# Is climate an important characteristic of major power outages in the United States?

**Name(s)**: Ripudh Mylapur, Chia Lee

**Website Link**: https://ripudhm.github.io/power-outages-analysis/

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

### Framing the Problem

In [2]:
import openpyxl
df = pd.read_excel(r"outage.xlsx", index_col = 1, header = 5)
df = df.drop(np.nan)
df = df.drop('variables', axis = 1)

def join_times(col_date, col_time):
    
    time = df[[col_date, col_time]]
    time = time.assign(date=pd.to_datetime(time[col_date]))
    time['date'] = time['date'].astype(str)
    time = time.assign(datetime=time['date'] + ' ' + time[col_time].astype(str))
    time = time.replace('NaT nan', np.nan)
    time = time.assign(fin=pd.to_datetime(time['datetime']))
    return time['fin']

out_start = join_times('OUTAGE.START.DATE', 'OUTAGE.START.TIME')
rest = join_times('OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME')
df = df.assign(out_start = out_start)
df = df.assign(rest_start = rest)
df = df.rename(columns = {'out_start': 'OUTAGE.START', 'rest_start': 'OUTAGE.RESTORATION'})


In [3]:
df.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

### Baseline Model

In [4]:
unique_regions = df['CLIMATE.REGION'].value_counts()
for reg in unique_regions.index:
    df[reg] = df['CLIMATE.REGION'].apply(lambda x: 1 if x == reg else 0)

In [5]:
df['CAUSE.CATEGORY']

OBS
1.0              severe weather
2.0          intentional attack
3.0              severe weather
4.0              severe weather
5.0              severe weather
                  ...          
1530.0            public appeal
1531.0    fuel supply emergency
1532.0                islanding
1533.0                islanding
1534.0        equipment failure
Name: CAUSE.CATEGORY, Length: 1534, dtype: object

In [6]:
df1 = df[['OUTAGE.DURATION', 'CUSTOMERS.AFFECTED','ANOMALY.LEVEL', 'MONTH','NERC.REGION','CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'POPULATION']]

In [7]:
df1 = df1.dropna()
df1

,OUTAGE.DURATION,CUSTOMERS.AFFECTED,ANOMALY.LEVEL,MONTH,NERC.REGION,CAUSE.CATEGORY,CLIMATE.CATEGORY,POPULATION
OBS,,,,,,,,
1.0,3060,70000.0,-0.3,7.0,MRO,severe weather,normal,5348119.0
3.0,3000,70000.0,-1.5,10.0,MRO,severe weather,cold,5310903.0
4.0,2550,68200.0,-0.1,6.0,MRO,severe weather,normal,5380443.0
5.0,1740,250000.0,1.2,7.0,MRO,severe weather,warm,5489594.0
6.0,1860,60000.0,-1.4,11.0,MRO,severe weather,cold,5310903.0
...,...,...,...,...,...,...,...,...
1523.0,95,35000.0,0.3,6.0,WECC,system operability disruption,normal,1391802.0
1524.0,360,0.0,-1.3,1.0,WECC,intentional attack,cold,1584134.0
1525.0,1548,0.0,-0.1,6.0,WECC,public appeal,normal,1363380.0


In [8]:
model = LinearRegression()
y = df1[['OUTAGE.DURATION']]
X = df1[['CUSTOMERS.AFFECTED']]
model.fit(X = X, y = y)

LinearRegression()

In [9]:
preproc = ColumnTransformer(
    transformers=[
    ('ohe', OneHotEncoder(), ['MONTH','NERC.REGION','CAUSE.CATEGORY', 'CLIMATE.CATEGORY']),    
#     ('standardise', StandardScaler(), ['CUSTOMERS.AFFECTED', 'ANOMALY.LEVEL'])
    ],
    remainder='passthrough' # Specify what to do with all other columns ('total_bill' here) – drop or passthrough.
)


In [10]:
pl_base = Pipeline([
    ('preproc', preproc),
    ('lin-reg', LinearRegression())
    ])

In [11]:
pl_base.fit(df1.drop(['OUTAGE.DURATION'], axis=1), df1['OUTAGE.DURATION'])

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['MONTH', 'NERC.REGION',
                                                   'CAUSE.CATEGORY',
                                                   'CLIMATE.CATEGORY'])])),
                ('lin-reg', LinearRegression())])

In [12]:
pl_base.score(df1.drop(['OUTAGE.DURATION'], axis=1), df1['OUTAGE.DURATION'])

0.06895656721070198

In [13]:
def rmse(actual, pred):
    return np.sqrt(np.mean((actual - pred) ** 2))

In [14]:
all_preds = model.predict(X)
rmse(df1['OUTAGE.DURATION'], all_preds.reshape(-1))

4282.432892427123

In [15]:
all_preds.reshape(-1)

array([2546.39007736, 2546.39007736, 2539.12560128, ..., 2263.88267403,
       2263.88267403, 2403.11846567])

### Final Model

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df1.drop(['OUTAGE.DURATION'], axis=1), df1['OUTAGE.DURATION'])

In [29]:
preproc = ColumnTransformer(
    transformers=[
    ('ohe', OneHotEncoder(), ['MONTH','NERC.REGION','CAUSE.CATEGORY', 'CLIMATE.CATEGORY']),    
    ('standardise', StandardScaler(), ['CUSTOMERS.AFFECTED', 'POPULATION'])
    ],
    remainder='passthrough' # Specify what to do with all other columns ('total_bill' here) – drop or passthrough.
)

In [30]:
pl1 = Pipeline([
    ('preproc', preproc),
    ('lin-reg', LinearRegression())
    ])

In [31]:
pl1.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['MONTH', 'NERC.REGION',
                                                   'CAUSE.CATEGORY',
                                                   'CLIMATE.CATEGORY']),
                                                 ('standardise',
                                                  StandardScaler(),
                                                  ['CUSTOMERS.AFFECTED',
                                                   'POPULATION'])])),
                ('lin-reg', LinearRegression())])

In [32]:
pl1.score(X_train, y_train)

0.32769561433169314

In [33]:
pl1.score(X_test, y_test)

-0.1082208251845731

In [34]:
polyreg = Pipeline([
    ('preproc', preproc),
    ('poly', PolynomialFeatures(1)),
    ('lin-reg', LinearRegression())
    ]
)

In [35]:
polyreg.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['MONTH', 'NERC.REGION',
                                                   'CAUSE.CATEGORY',
                                                   'CLIMATE.CATEGORY']),
                                                 ('standardise',
                                                  StandardScaler(),
                                                  ['CUSTOMERS.AFFECTED',
                                                   'POPULATION'])])),
                ('poly', PolynomialFeatures(degree=1)),
                ('lin-reg', LinearRegression())])

In [36]:
polyreg.score(X_train, y_train)

0.32769561433194117

In [37]:
polyreg.score(X_test, y_test)

-0.10822079949828622

In [38]:
def polynomialreg(degree=2):
    return Pipeline([
    ('preproc', preproc),
    ('poly', PolynomialFeatures(degree)),
    ('lin-reg', LinearRegression())
    ]
)

In [39]:
hyperparameters = {'poly__degree': [1,2,3]}

In [40]:
searcher = GridSearchCV(polyreg, hyperparameters, cv=5)

In [41]:
searcher.fit(X_train, y_train)

C:\Users\ripud\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\utils\metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\pipeline.py", line 707, in score
    Xt = transform.transform(Xt)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py", line 748, in transform
    Xs = self._fit_trans

C:\Users\ripud\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\utils\metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\pipeline.py", line 707, in score
    Xt = transform.transform(Xt)
  File "C:\Users\ripud\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py", line 748, in transform
    Xs = self._fit_trans

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(),
                                                                         ['MONTH',
                                                                          'NERC.REGION',
                                                                          'CAUSE.CATEGORY',
                                                                          'CLIMATE.CATEGORY']),
                                                                        ('standardise',
                                                                         StandardScaler(),
                                                                         ['CUSTOMERS.AFFECTED',
                                          

In [42]:
searcher.best_params_

{'poly__degree': 1}

### Fairness Analysis

In [43]:
df2 = df[['OUTAGE.DURATION', 'CUSTOMERS.AFFECTED','ANOMALY.LEVEL', 'MONTH','NERC.REGION','CAUSE.CATEGORY', 'CLIMATE.CATEGORY','POPULATION', 'U.S._STATE']]
df2['is_Cali'] = df2['U.S._STATE'].apply(lambda x: 1 if x=='California' else 0)
df2= df2.dropna()
df2['is_Cali'].sum()

C:\Users\ripud\AppData\Local\Temp\ipykernel_23676\243506506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['is_Cali'] = df2['U.S._STATE'].apply(lambda x: 1 if x=='California' else 0)


121

In [44]:
df2

,OUTAGE.DURATION,CUSTOMERS.AFFECTED,ANOMALY.LEVEL,MONTH,NERC.REGION,CAUSE.CATEGORY,CLIMATE.CATEGORY,POPULATION,U.S._STATE,is_Cali
OBS,,,,,,,,,,
1.0,3060,70000.0,-0.3,7.0,MRO,severe weather,normal,5348119.0,Minnesota,0
3.0,3000,70000.0,-1.5,10.0,MRO,severe weather,cold,5310903.0,Minnesota,0
4.0,2550,68200.0,-0.1,6.0,MRO,severe weather,normal,5380443.0,Minnesota,0
5.0,1740,250000.0,1.2,7.0,MRO,severe weather,warm,5489594.0,Minnesota,0
6.0,1860,60000.0,-1.4,11.0,MRO,severe weather,cold,5310903.0,Minnesota,0
...,...,...,...,...,...,...,...,...,...,...
1523.0,95,35000.0,0.3,6.0,WECC,system operability disruption,normal,1391802.0,Idaho,0
1524.0,360,0.0,-1.3,1.0,WECC,intentional attack,cold,1584134.0,Idaho,0
1525.0,1548,0.0,-0.1,6.0,WECC,public appeal,normal,1363380.0,Idaho,0


In [45]:
y_pred = pl1.predict(df2)

In [46]:
df2['prediction'] = y_pred

In [47]:
df2

,OUTAGE.DURATION,CUSTOMERS.AFFECTED,ANOMALY.LEVEL,MONTH,NERC.REGION,CAUSE.CATEGORY,CLIMATE.CATEGORY,POPULATION,U.S._STATE,is_Cali,prediction
OBS,,,,,,,,,,,
1.0,3060,70000.0,-0.3,7.0,MRO,severe weather,normal,5348119.0,Minnesota,0,2511.695818
3.0,3000,70000.0,-1.5,10.0,MRO,severe weather,cold,5310903.0,Minnesota,0,4775.488529
4.0,2550,68200.0,-0.1,6.0,MRO,severe weather,normal,5380443.0,Minnesota,0,2834.880003
5.0,1740,250000.0,1.2,7.0,MRO,severe weather,warm,5489594.0,Minnesota,0,2726.061025
6.0,1860,60000.0,-1.4,11.0,MRO,severe weather,cold,5310903.0,Minnesota,0,1926.285831
...,...,...,...,...,...,...,...,...,...,...,...
1523.0,95,35000.0,0.3,6.0,WECC,system operability disruption,normal,1391802.0,Idaho,0,-550.839055
1524.0,360,0.0,-1.3,1.0,WECC,intentional attack,cold,1584134.0,Idaho,0,950.398633
1525.0,1548,0.0,-0.1,6.0,WECC,public appeal,normal,1363380.0,Idaho,0,2229.243192


In [48]:
def rmse(actual, pred):
    return np.sqrt(np.mean((actual - pred) ** 2))

In [49]:
df2['act-pred^2'] = (df2['OUTAGE.DURATION'] - df2['prediction']) ** 2
df2

,OUTAGE.DURATION,CUSTOMERS.AFFECTED,ANOMALY.LEVEL,MONTH,NERC.REGION,CAUSE.CATEGORY,CLIMATE.CATEGORY,POPULATION,U.S._STATE,is_Cali,prediction,act-pred^2
OBS,,,,,,,,,,,,
1.0,3060,70000.0,-0.3,7.0,MRO,severe weather,normal,5348119.0,Minnesota,0,2511.695818,300637.47632
3.0,3000,70000.0,-1.5,10.0,MRO,severe weather,cold,5310903.0,Minnesota,0,4775.488529,3152359.514908
4.0,2550,68200.0,-0.1,6.0,MRO,severe weather,normal,5380443.0,Minnesota,0,2834.880003,81156.615921
5.0,1740,250000.0,1.2,7.0,MRO,severe weather,warm,5489594.0,Minnesota,0,2726.061025,972316.345781
6.0,1860,60000.0,-1.4,11.0,MRO,severe weather,cold,5310903.0,Minnesota,0,1926.285831,4393.811349
...,...,...,...,...,...,...,...,...,...,...,...,...
1523.0,95,35000.0,0.3,6.0,WECC,system operability disruption,normal,1391802.0,Idaho,0,-550.839055,417108.085079
1524.0,360,0.0,-1.3,1.0,WECC,intentional attack,cold,1584134.0,Idaho,0,950.398633,348570.545894
1525.0,1548,0.0,-0.1,6.0,WECC,public appeal,normal,1363380.0,Idaho,0,2229.243192,464092.287292


In [50]:
obs = np.sqrt(df2.groupby('is_Cali')['act-pred^2'].mean()).diff().iloc[-1]
obs

931.2386427883148

In [51]:
diff_in_rmse = []
for _ in range(100):
    s = (
        np.sqrt(df2[['is_Cali', 'prediction', 'OUTAGE.DURATION', 'act-pred^2']]
        .assign(is_Cali=df2.is_Cali.sample(frac=1.0, replace=False).reset_index(drop=True))
        .groupby('is_Cali')['act-pred^2']
        .mean())
        .diff()
        .iloc[-1]
    )
    
    diff_in_rmse.append(s)

In [52]:
np_diff_rmse = np.array(diff_in_rmse)
(obs < np_diff_rmse).sum() / len(diff_in_rmse)

0.12

In [53]:
fig = pd.Series(diff_in_rmse).plot(kind='hist', histnorm='probability', nbins=20,
                            title='Difference in RMSE (Cali - Not Cali)')
fig.add_vline(x=obs, line_color='red')
fig.write_html('diff_rmse.html', include_plotlyjs='cdn')